In [1]:
from gwg_utils import *
import pandas as pd
import finplot as fplt

In [2]:
for symbol in ['QQQ', 'IWM', 'SPY']:
    select_sql = f"""
        with quote_ta as (
            select 
                q.date,q.symbol, q.open,q.close,q.high,q.low,q.volume,
                t.ema15, t.ema50, t.ema200,
                t.macd, t.sig, t.diff, t.trend
            from quote_daily q, quote_ta t
            where q.date = t.date
            and q.symbol = t.symbol
            and t.symbol = '{symbol}'
            order by q.date desc
            limit {QUOTE_TA_LIMIT}
        )
        select * from quote_ta
        order by date
        ;

    """
    df = pd.read_sql(select_sql, engine)

    df["date"] = pd.to_datetime(df['date']).astype('int64')

    #df2.columns
    #df["date"] = df.index.strftime("%Y-%m-%d")
    #df.set_index('date')

    # df.to_csv("qqq-ta-test.csv", index=False)
    # df2 = pd.read_csv("qqq-ta-test.csv")

    ax,ax2 = fplt.create_plot(f'{symbol}', rows=2)

    fplt.candlestick_ochl(df[['date','open','close','high','low']], ax=ax)
    fplt.plot(df[["ema15"]], ax=ax)
    fplt.plot(df[["ema50"]], ax=ax, width=2)
    fplt.plot(df[["ema200"]], ax=ax, width=3)

    # axo = ax.overlay()
    # fplt.volume_ocv(df[['date','open','close','volume']], ax=axo)

    fplt.plot(df[["macd"]], ax=ax2, width=2, legend='MACD')
    fplt.plot(df[["sig"]], ax=ax2, legend='Signal')
    fplt.plot(df[["diff"]], ax=ax2, legend='Diff')
    fplt.plot(2*df[["trend"]], ax=ax2, width=2, legend='Trend')
    fplt.volume_ocv(df[['date','open','close','diff']], ax=ax2, colorfunc=fplt.strength_colorfilter)

    fplt.show()